Simple & Multi Linear Regression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls /content/drive/MyDrive/Colab\ Notebooks/'HOPE AI'/WEEK-3-ML/1.Traditional_way/6.Baby_Assignment/

6_Baby_Assignment.ipynb  insurance_pre.csv


In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/HOPE AI/WEEK-3-ML/1.Traditional_way/6.Baby_Assignment/insurance_pre.csv")

In [ ]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [ ]:
dataset.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges'], dtype='object')

In [ ]:
dataset = pd.get_dummies(dataset,dtype=int,drop_first=1)

In [ ]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [ ]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [ ]:
independent = dataset[['age', 'sex_male', 'bmi', 'children', 'smoker_yes']]

In [ ]:
dependent = dataset[['charges']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)

LinearRegression()

In [ ]:
weight=regressor.coef_
weight

array([[  257.8006705 ,   -41.74825718,   321.06004271,   469.58113407,
        23418.6671912 ]])

In [ ]:
bais=regressor.intercept_
bais

array([-12057.244846])

In [ ]:
y_pred=regressor.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)

In [ ]:
r_score

0.789479034986701

Support Vector Regression

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from itertools import product

sc = StandardScaler()
X_train_svm = sc.fit_transform(x_train)
X_test_svm = sc.transform(x_test)

#comomn c_value parameter
#commong kernels for all
C_values = [10, 100, 500, 1000, 2000, 3000]
kernels = ['linear', 'rbf', 'poly', 'sigmoid']

#loop
svm_results = []
for C, kernel in product(C_values, kernels):
    reg = SVR(C=C, kernel=kernel)
    reg.fit(X_train_svm, y_train.values.ravel())
    r2 = r2_score(y_test, reg.predict(X_test_svm))
    svm_results.append([C, kernel, r2])

svm_df = pd.DataFrame(svm_results, columns=["C", "Kernel", "R²"])
print("\nSUPPORT VECTOR MACHINE RESULTS")
print(svm_df.pivot(index="C", columns="Kernel", values="R²").round(4))

best_svm = svm_df.loc[svm_df['R²'].idxmax()]
print(f"Best SVM: C={best_svm.C}, Kernel={best_svm.Kernel}, R²={best_svm['R²']:.4f}")


SUPPORT VECTOR MACHINE RESULTS
Kernel  linear    poly     rbf  sigmoid
C                                      
10      0.4625  0.0387 -0.0323   0.0393
100     0.6289  0.6180  0.3200   0.5276
500     0.7631  0.8264  0.6643   0.4446
1000    0.7649  0.8566  0.8102   0.2875
2000    0.7440  0.8606  0.8548  -0.5940
3000    0.7414  0.8599  0.8663  -2.1244
Best SVM: C=3000, Kernel=rbf, R²=0.8663


Decision Tree

In [ ]:
# Decision tree
from sklearn.tree import DecisionTreeRegressor

criteria = ['squared_error', 'absolute_error', 'friedman_mse']
max_features = ['sqrt', 'log2']
splitters = ['best', 'random']

dt_results = []
for crit, feat, split in product(criteria, max_features, splitters):
    reg = DecisionTreeRegressor(criterion=crit, splitter=split, max_features=feat, random_state=0)
    reg.fit(x_train, y_train)
    r2 = r2_score(y_test, reg.predict(x_test))
    dt_results.append([crit, feat, split, r2])

dt_df = pd.DataFrame(dt_results, columns=['Criterion', 'Max_Features', 'Splitter', 'R²'])
print("\nDECISION TREE RESULTS")
print(dt_df.pivot_table(index=['Criterion', 'Max_Features'], columns='Splitter', values='R²').round(4))

best_dt = dt_df.loc[dt_df['R²'].idxmax()]
print(f"Best Decision Tree: {best_dt.to_dict()}")


DECISION TREE RESULTS
Splitter                       best  random
Criterion      Max_Features                
absolute_error log2          0.7498  0.6754
               sqrt          0.7498  0.6754
friedman_mse   log2          0.7215  0.5942
               sqrt          0.7215  0.5942
squared_error  log2          0.7215  0.5948
               sqrt          0.7215  0.5948
Best Decision Tree: {'Criterion': 'absolute_error', 'Max_Features': 'sqrt', 'Splitter': 'best', 'R²': 0.7497857021005525}


Random Forest

In [ ]:
#Random forest
from sklearn.ensemble import RandomForestRegressor

criteria = ['squared_error', 'absolute_error']

max_features = ['sqrt', 'log2']
n_estimators = [10, 100]

rf_results = []
for crit, feat, n in product(criteria, max_features, n_estimators):
    reg = RandomForestRegressor(criterion=crit, max_features=feat, n_estimators=n, random_state=0)
    reg.fit(x_train, y_train.values.ravel())
    r2 = r2_score(y_test, reg.predict(x_test))
    rf_results.append([crit, feat, n, r2])

rf_df = pd.DataFrame(rf_results, columns=['Criterion', 'Max_Features', 'N_Estimators', 'R²'])
print("\nRANDOM FOREST RESULTS")
print(rf_df.pivot_table(index=['Criterion', 'Max_Features'], columns='N_Estimators', values='R²').round(4))

best_rf = rf_df.loc[rf_df['R²'].idxmax()]
print(f"Best Random Forest: {best_rf.to_dict()}")


RANDOM FOREST RESULTS
N_Estimators                    10      100
Criterion      Max_Features                
absolute_error log2          0.8531  0.8686
               sqrt          0.8531  0.8686
squared_error  log2          0.8484  0.8678
               sqrt          0.8484  0.8678
Best Random Forest: {'Criterion': 'absolute_error', 'Max_Features': 'sqrt', 'N_Estimators': 100, 'R²': 0.868570010750439}


In [ ]:

# SUMMARY OF ALL MODELS
summary_data = [
    ['Multiple Linear Regression', r_score],
    ['SVM', best_svm['R²']],
    ['Decision Tree', best_dt['R²']],
    ['Random Forest', best_rf['R²']]
]

summary = pd.DataFrame(summary_data, columns=['Model', 'Best_R2'])

print('\nALL MODELS PERFORMANCE:')
for index, row in summary.iterrows():
    print(f"{row['Model']}: {row['Best_R2']:.4f}")

best_overall = summary.loc[summary['Best_R2'].idxmax()]
print('\n' + '='*30)
print(f"FINAL BEST MODEL: {best_overall['Model']}")
print(f"Best R2 Score: {best_overall['Best_R2']:.4f}")
print('='*30)


ALL MODELS PERFORMANCE:
Multiple Linear Regression: 0.7895
SVM: 0.8663
Decision Tree: 0.7498
Random Forest: 0.8686

FINAL BEST MODEL: Random Forest
Best R2 Score: 0.8686
